In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# data paths
small_path = '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small'
full_path = '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest'

small:
- links.csv (movieId,imdbId,tmdbId)
- movies.csv (movieId,title,genres)
- ratings.csv (userId,movieId,rating,timestamp)
- tags.csv (userId,movieId,tag,timestamp)


full:
- links.csv
- movies.csv
- ratings.csv
- tags.csv
- genome-scores.csv
- genome-tags.csv

## ml-latest-small

In [3]:
# import ratings from small dataset
ratings_small = pd.read_csv(small_path + '/ratings.csv')
print(ratings_small.shape)
ratings_small.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# stratify split
train_small, test_val_small = train_test_split(ratings_small, stratify=ratings_small['userId'], test_size=0.2)
test_small, val_small = train_test_split(test_val_small, stratify=test_val_small['userId'], test_size=0.5)

In [5]:
train_small.head

<bound method NDFrame.head of        userId  movieId  rating   timestamp
23635     160     4370     4.0  1065992012
99816     610     4270     3.0  1493845935
37109     249    85780     4.0  1368891309
74536     474     6042     3.0  1069685537
72065     464      653     2.5  1287400499
...       ...      ...     ...         ...
54542     358    81591     2.0  1339539299
47846     309     1198     4.0  1166068294
78757     489     2628     1.0  1333101402
20303     134       48     3.0   832841524
7944       54      318     4.0   830247358

[80668 rows x 4 columns]>

In [6]:
test_small.head

<bound method NDFrame.head of        userId  movieId  rating   timestamp
69527     448     6503     1.0  1105009312
66096     425     3948     3.5  1085490893
65298     419       47     4.0  1321659049
46041     305     1253     5.0  1460366849
51818     335      466     3.0  1261541271
...       ...      ...     ...         ...
10120      66     1862     1.0  1113190775
13189      84      527     5.0   857653594
63782     414     4818     2.0  1091716530
35897     244     1228     4.0   975075169
37746     256     1270     5.0  1447001868

[10084 rows x 4 columns]>

In [7]:
val_small.head

<bound method NDFrame.head of        userId  movieId  rating   timestamp
89689     580     8950     4.5  1167789905
48183     312     3701     3.0  1043176884
79331     492      619     3.0   863976722
77116     482     1037     3.5  1105396623
47612     307    48518     2.0  1189608059
...       ...      ...     ...         ...
12350      76    34405     5.0  1439165874
21675     140    43396     4.0  1166645387
77578     483     3785     3.0  1215896116
9020       62    86880     4.0  1523786545
63774     414     4776     4.0  1017669912

[10084 rows x 4 columns]>

In [8]:
# Check that stratification worked
print('Number of unique users in original dataset:', ratings_small['userId'].nunique())
print('Number of unique users in train dataset:', train_small['userId'].nunique())
print('Number of unique users in test dataset:', test_small['userId'].nunique())
print('Number of unique users in val dataset:', val_small['userId'].nunique())

Number of unique users in original dataset: 610
Number of unique users in train dataset: 610
Number of unique users in test dataset: 610
Number of unique users in val dataset: 610


In [20]:
# save to csv!
train_small.to_csv('ratings_small_train.csv')
test_small.to_csv('ratings_small_test.csv')
val_small.to_csv('ratings_small_val.csv')

## ml-latest

In [3]:
# repeat for full dataset
# Note: selected users have at least 1 movie 

# import ratings from full dataset
#ratings_full = pd.read_csv(full_path + '/ratings.csv')
#print(ratings_full.shape)
#ratings_full.head()

(27753444, 4)


,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


When we tried to split the full dataframe based on the number of counts, it caused the kernel to die when using both pandas and Dask. So we split the data using Spark instead. We will now import these 2 csv files, use sklearn to split the file of ratings for users who have 3 or more ratings, and append the training set to the users who have 2 or less ratings. 

In [3]:
df_toStratify = pd.read_csv('full_tostratify.csv', names=['userId', 'movieId', 'rating', 'timestamp', 'count'])
df_lessThan10 = pd.read_csv('full_lessthan10.csv', names=['userId', 'movieId', 'rating', 'timestamp', 'count'])

In [4]:
# split stratify one normally
train_all, test_val_all = train_test_split(df_toStratify, stratify=df_toStratify['userId'], test_size=0.2)

In [5]:
test_all, val_all = train_test_split(test_val_all, stratify=test_val_all['userId'], test_size=0.5)

Now handle ones that couldn't be stratified normally

In [6]:
# handles 1s and 2s
train_1and2s = df_lessThan10[df_lessThan10['count'] < 3] # append to training later

In [7]:
train_1and2s

,userId,movieId,rating,timestamp,count
28,6357,2968,3.0,1327051222,1
29,7340,54286,3.5,1482613075,1
63,16386,114601,4.0,1503233667,1
88,19079,145,3.0,1303292120,2
89,19079,1909,2.0,1303292125,2
...,...,...,...,...,...
187589,266949,1131,5.0,886001029,2
187590,266949,1235,5.0,886001029,2
187591,267468,834,3.0,962893364,2
187592,267468,1029,1.0,962893364,2


In [8]:
# handle 3s to 9s
toStratify_3to9s = df_lessThan10[df_lessThan10['count'] >= 3]

In [9]:
toStratify_3to9s

,userId,movieId,rating,timestamp,count
0,1088,912,5.0,990053868,5
1,1088,1947,4.0,990053868,5
2,1088,1989,1.0,990053868,5
3,1088,2144,4.0,990053876,5
4,1088,2712,2.0,990053876,5
...,...,...,...,...,...
187665,283166,1356,2.0,962788794,7
187666,283166,2398,1.0,962788756,7
187667,283166,2628,4.0,962788794,7
187668,283166,3087,1.0,962788466,7


In [10]:
# split these into 3rds
train_3to9s, test_val_3to9s = train_test_split(toStratify_3to9s, stratify=toStratify_3to9s['userId'], test_size=0.67)
test_3to9s, val_3to9s = train_test_split(test_val_3to9s, stratify=test_val_3to9s['userId'], test_size=0.5)

In [24]:
print(train_3to9s.shape)
print(test_3to9s.shape)
print(val_3to9s.shape)

(57468, 5)
(58340, 5)
(58340, 5)


Now we just need to drop the counts column before writing these to csv files

In [26]:
train_all.drop(columns=['count'], inplace=True)
test_all.drop(columns=['count'], inplace=True)
val_all.drop(columns=['count'], inplace=True)

train_1and2s.drop(columns=['count'], inplace=True)
train_3to9s.drop(columns=['count'], inplace=True)
test_3to9s.drop(columns=['count'], inplace=True)
val_3to9s.drop(columns=['count'], inplace=True)

/ext3/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
# check
val_3to9s

,userId,movieId,rating,timestamp
182738,213324,1196,4.0,993751936
46569,233773,134130,4.0,1453174538
180131,216242,1213,2.0,1493077978
158787,232035,3510,5.0,1174417803
115321,251536,2762,3.0,981211823
...,...,...,...,...
56735,68721,191351,3.0,1537080080
51657,19029,86880,2.0,1338842345
45794,33298,58559,5.0,1494789405
74374,157292,1393,3.0,901131437


In [34]:
# Try writing and appending
# first write train/val/test_all
train_all.to_csv('ratings_all_train.csv',header ='column_names')

In [35]:
test_all.to_csv('ratings_all_test.csv',header ='column_names')

In [36]:
val_all.to_csv('ratings_all_val.csv',header ='column_names')

In [37]:
# Now try appending others
# train
train_1and2s.to_csv('ratings_all_train.csv', mode='a', header=False)
train_3to9s.to_csv('ratings_all_train.csv', mode='a', header=False)

# test
test_3to9s.to_csv('ratings_all_test.csv', mode='a', header=False)

# val
val_3to9s.to_csv('ratings_all_val.csv', mode='a', header=False)

Now check proportions:

In [38]:
train_size = train_all.shape[0] + train_1and2s.shape[0] + train_3to9s.shape[0]
test_size = test_all.shape[0] + test_3to9s.shape[0]
val_size = val_all.shape[0] + val_3to9s.shape[0]

print('train size:', train_size)
print('test size:', test_size)
print('val size:', val_size)

train size: 22123609
test size: 2814917
val size: 2814918


In [39]:
# check that these match original size
total_size = train_size + test_size + val_size
print(total_size == df_toStratify.shape[0] + df_lessThan10.shape[0])
print('total rows:', total_size) #27753444, which we know is correct!!!

True
total rows: 27753444


In [40]:
# Calculate percentages for train, text, val, to verify
print('Train proportion:', round(train_size * 100 / total_size, 3), '%')
print('Test proportion:', round(test_size * 100 / total_size,3), '%')
print('Val proportion:', round(val_size * 100 / total_size,3), '%')

Train proportion: 79.715 %
Test proportion: 10.143 %
Val proportion: 10.143 %


Even with our handling of records with too few counts to stratify, we were able to achieve very close to our desired 80-10-10 split!